# Scikit + EC2

Recommended instance type: c5.4xlarge

In [2]:
!aws s3 cp s3://rikturr/2015_partB_sparse.npz .
!aws s3 cp s3://rikturr/2015_partB_lookup.csv .

download: s3://rikturr/2015_partB_sparse.npz to ./2015_partB_sparse.npz
download: s3://rikturr/2015_partB_lookup.csv to ./2015_partB_lookup.csv


In [8]:
!pip install scikit-learn

    100% |████████████████████████████████| 12.4MB 123kB/s  eta 0:00:01


In [3]:
import scipy.sparse as sp
import pandas as pd
import numpy as np

random_state = 42
labels = pd.read_csv('2015_partB_lookup.csv')
features = sp.load_npz('2015_partB_sparse.npz')

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #2
  (fname, cnt))
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/matplotlib/__init__.py:962: UserWarning: Duplicate key in file "/home/ec2-user/.config/matplotlib/matplotlibrc", line #3
  (fname, cnt))


In [4]:
labels.head()

,npi,provider_type
0,1003000126,Internal Medicine
1,1003000142,Anesthesiology
2,1003000407,Family Practice
3,1003000522,Family Practice
4,1003000530,Internal Medicine


In [5]:
features

<516476x4206 sparse matrix of type '<class 'numpy.float64'>'
	with 5596950 stored elements in Compressed Sparse Column format>

In [10]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(features, 
                                                    labels['provider_type'], 
                                                    test_size=0.3, 
                                                    random_state=random_state)

In [11]:
x_train.shape

(361533, 4206)

In [12]:
y_train.shape

(361533,)

In [14]:
%%time
from sklearn.ensemble import RandomForestClassifier
from multiprocessing import cpu_count

rf = RandomForestClassifier(n_estimators=100, n_jobs=cpu_count(), random_state=random_state)
rf.fit(x_train, y_train)

CPU times: user 50min 5s, sys: 1.87 s, total: 50min 7s
Wall time: 3min 20s


In [22]:
from sklearn.metrics import classification_report

predicted = rf.predict(x_test)
print(classification_report(y_test, predicted))

                      precision    recall  f1-score   support

      Anesthesiology       0.98      0.98      0.98     10030
          Cardiology       0.91      0.91      0.91      6206
Diagnostic Radiology       0.99      0.99      0.99      8716
  Emergency Medicine       0.81      0.84      0.82     12199
     Family Practice       0.54      0.58      0.56     24208
   Internal Medicine       0.68      0.67      0.68     29214
  Nurse Practitioner       0.52      0.57      0.55     23458
       Ophthalmology       0.99      0.98      0.99      5181
  Orthopedic Surgery       0.81      0.88      0.84      6338
  Physical Therapist       1.00      1.00      1.00     13009
 Physician Assistant       0.51      0.36      0.42     16384

         avg / total       0.72      0.72      0.71    154943



In [23]:
pd.crosstab(y_test, predicted)

col_0,Anesthesiology,Cardiology,Diagnostic Radiology,Emergency Medicine,Family Practice,Internal Medicine,Nurse Practitioner,Ophthalmology,Orthopedic Surgery,Physical Therapist,Physician Assistant
provider_type,,,,,,,,,,,
Anesthesiology,9802,2,2,6,19,52,103,0,16,2,26
Cardiology,3,5664,7,3,26,289,165,1,1,0,47
Diagnostic Radiology,6,11,8625,3,14,10,23,1,1,0,22
Emergency Medicine,8,4,9,10269,536,216,393,1,17,0,746
Family Practice,6,18,28,552,14068,5117,3404,5,135,0,875
Internal Medicine,44,418,15,278,5382,19701,2757,5,65,8,541
Nurse Practitioner,41,58,12,565,3847,2240,13467,10,239,8,2971
Ophthalmology,1,0,0,2,7,5,61,5087,1,1,16
Orthopedic Surgery,22,2,3,6,23,58,274,1,5592,8,349
